# Bootstrapping

In [ ]:
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
%matplotlib inline

## Sampling with vs. without Replacement

Many questions of probability and statistics have to do with sampling from a population. Sometimes it is appropriate to imagine that the initial population is reset after each draw, and sometimes it is not.

Here's a case of the former: I go around interviewing people and I record their birthdays.

Clearly, two people can have the same birthday, and so I need each birthday to be available for every draw (i.e. interview). This is sampling **with replacement**. (One might even imagine that I interview some people more than once (by random chance).) Note in particular that draws in this context are mutually **independent**.

Here's a case of the latter: I am dealt thirteen cards to make a bridge hand. I can't have the same card appear twice in a single hand, so if I am thinking about the statistics of bridge hands, then I'm thinking about sampling **without replacement**.

Clearly this difference has an effect on correct calculation.

Consider these two similar cases:

**Case 1**: We're playing war. Each of us has a deck of cards, and we each turn over one card at a time, the player with the higher card collecting both. A tie in rank triggers "war", where more cards are laid down and then another contest is initiated on top of the now larger "pot" of cards.

Question: What are the chances that you and I turn over cards with the same rank on one round of war?

Answer: This is effectively a problem of cards drawn *with replacement*: I can just model this with two draws from a single deck. And so I calculate as follows:

I can draw any card first, so that's 52/52. The second card must match the first in rank, so that's 4/52. Thus the chances are $\frac{52}{52}\times\frac{4}{52} = \frac{1}{13}$.

**Case 2**: I am dealt a two-card blackjack hand from a single deck. (Good luck finding this game in Las Vegas!)

Question: What are the chances that I am dealt a pair?

Answer: The two events, one for each card being passed my way, are *not independent*. What the second card is likely to be is affected by what the first card is. For example, if the first card dealt me is the ace of spades, then there is now zero chance that the second card dealt me will be the ace of spades. (Whereas, if the first card is something else, then the chance that the second card be the ace of spades is greater than 0.) And so this is a problem of cards drawn *without replacement*.

So I calculate as follows:

I can draw any card first, so that's 52/52. The second card must match the first in rank, so that's 3/51. There are three left of whatever rank matches my first card, and 51 total cards left. Thus the chances are $\frac{52}{52}\times\frac{3}{51} = \frac{1}{17}$.

Bootstrapping is a sort of sampling ***with replacement***.

I can effect both sorts of sampling with the `choice()` function inside NumPy's `random` module:

In [ ]:
X = ['red', 'green', 'blue']
np.random.choice(X)

Let's check the defaults of this function!

In [ ]:
np.random.choice()

## Sampling from an Unknown Distribution

Bootstrapping is used when the shape of the population's distribution is unknown. To simulate this situation, let's make several distributions:

In [ ]:
norm = stats.norm(loc=10, scale=5)
expon = stats.expon(loc=5, scale=5)
uni = stats.uniform(loc=5, scale=10)

dists = [norm, expon, uni]

Note that these distributions all have the same mean:

In [ ]:
norm.mean() == expon.mean() == uni.mean() == 10

Plotting:

In [ ]:
def plot_dists(dists, n=100):
    """Plot histograms of the distributions in dists."""
    fig, axs = plt.subplots(1, len(dists), figsize=(5*len(dists), 5))
    for ax, dist in zip(axs, dists):
        ax.hist(dist.rvs(10000))
        ax.set_xlim(0, 30)

In [ ]:
plot_dists(dists)

Choose a distribution at random:

In [ ]:
dist = np.random.choice(dists)

Take a million points from it:

In [ ]:
np.random.seed(42)
pop = dist.rvs(10**6)

Take a sample of 1000 from that million:

In [ ]:
sample = np.random.choice(pop, size=1000)

In [ ]:
len(sample)

In [ ]:
sample.mean()

The sample mean, $\bar{x}$, is *near* the population mean, $\mu$, but there's a certain gap between them.

## The Algorithm in Words

What we want to do now is what we just did, but *many* times. We can record statistics about each sample, and **then** do statistics on those statistics!

The idea is that statistics on this collection of samples, each made **with replacement**, will be a good approximation of the population parameters from which our original sample was drawn. And the more samples we take, the better our approximation should be. In this way we are "pulling ourselves up by our own bootstraps" to make inferences about the population as a whole.

**Note that we are NOT making inferences about the population distribution, but only about some population parameter of interest (like the mean).**

Let's see what happens when we record the mean and the 95th percentile of each sample:

In [ ]:
np.random.seed(1)

# Initialize an empty list
bootstrap_samples = []

# Initialize an array of means
bootstrap_sample_means = np.zeros(1000)

# And another of 95th percentiles
bootstrap_sample_95pcts = np.zeros(1000)
for i in range(1000):
    
    # Take 1000 points from one of the dists
    bootstrap_sample = np.random.choice(sample, size=1000)
    
    # Add that to the list
    bootstrap_samples.append(bootstrap_sample)
    
    # Add the mean to the means' array
    bootstrap_sample_means[i] = bootstrap_sample.mean()
    
    # Add the 95th percentile to the percentiles'array
    bootstrap_sample_95pct = np.percentile(a=bootstrap_sample, q=95)
    bootstrap_sample_95pcts[i] = bootstrap_sample_95pct

In [ ]:
bootstrap_sample_means[:10]

In [ ]:
bootstrap_sample_95pcts[:10]

In [ ]:
fig, ax = plt.subplots(2, figsize=(6, 10))
ax[0].hist(bootstrap_sample_means)
ax[1].hist(bootstrap_sample_95pcts);

In [ ]:
sample.mean()

In [ ]:
bootstrap_sample_means.mean()

In [ ]:
pop.mean()

In [ ]:
np.percentile(a=sample, q=95)

In [ ]:
bootstrap_sample_95pcts.mean()

In [ ]:
np.percentile(a=pop, q=95)

## Why Bootstrap?

[Wikipedia](https://en.wikipedia.org/wiki/Bootstrapping_(statistics)) is helpful on this.

With a bootstrap we are simulating the relationship between population and sample by treating our sample as the population. In that case we can actually measure the error between our estimates (made through resampling) and the true sample statistics.

"Adèr et al. recommend the bootstrap procedure for the following situations (Adèr, H. J., Mellenbergh G. J., & Hand, D. J. (2008). *Advising on research methods: A consultant's companion*. Huizen, The Netherlands: Johannes van Kessel Publishing. ISBN 978-90-79418-01-5.):

- When the theoretical distribution of a statistic of interest is complicated or unknown. Since the bootstrapping procedure is distribution-independent it provides an indirect method to assess the properties of the distribution underlying the sample and the parameters of interest that are derived from this distribution.
- When the sample size is insufficient for straightforward statistical inference. If the underlying distribution is well-known, bootstrapping provides a way to account for the distortions caused by the specific sample that may not be fully representative of the population.
- When power calculations have to be performed, and a small pilot sample is available. Most power and sample size calculations are heavily dependent on the standard deviation of the statistic of interest. If the estimate used is incorrect, the required sample size will also be wrong. One method to get an impression of the variation of the statistic is to use a small pilot sample and perform bootstrapping on it to get impression of the variance."

## Bootstrapping as Comparison Tool

In the context of hypothesis testing, we can use bootstrapping to test whether there is a significant difference between two samples. The idea is this:

Suppose that Sample A and Sample B seem to be significantly different in some feature. Now, if we were (i) to throw A and B together into one big pool, and then (ii) to construct new samples (with replacement) from this pool, any similar difference we see between the samples we make would be evidence that the original difference between A and B was not significant after all. So if we do this sampling experiment lots of times, and we *rarely* see a difference between our two samples like we saw between A and B, then we can be confident that there is some significant difference between A and B.

Let's try this with the Instagram control and experimental groups:

In [ ]:
control = pd.read_csv('data/control.csv', index_col=0)
experiment = pd.read_csv('data/experiment.csv', index_col=0)

In [ ]:
def bootstrap(a, b):
    universe = np.append(a, b)
    universe_shuffled = np.random.choice(universe, size=len(universe), replace=True)
    new_a = universe_shuffled[:len(a)]
    new_b = universe_shuffled[len(a):]
    return new_a, new_b

In [ ]:
grp_1 = control['Likes_Given_Con']
grp_2 = experiment['Likes_Given_Exp']
orig_mean = grp_2.mean() - grp_1.mean()
orig_mean

In [ ]:
ctr = 0
samples = 10000
for _ in range(samples):
    a, b = bootstrap(grp_1, grp_2)
    if abs(a.mean() - b.mean()) > orig_mean:
        ctr += 1
print('p-value:' + str(ctr / samples))

### Relation to Permutation Tests

This use of bootstrapping as a comparision tool is similar in concept to the idea of a **permutation test**. The idea here is that, if some statistical test yields a high significance result (low p-value), then we should *not* see the same significance if we were to permute the labels of our data points, sampling randomly and indiscriminately from the control and the experimental groups.

The main difference is that bootstrapping always samples **with replacement** while permutation tests operate **without replacement** (as per the nature of permutations!).

#### Permutation Test Example

Let's check out the diamonds datset in `seaborn`.

In [ ]:
diamonds = sns.load_dataset('diamonds')

In [ ]:
sns.barplot(data=diamonds, x='clarity', y='price');

At least as far as this dataset is concerned, diamonds with a clarity rating of "SI2" seem to demand a significantly higher price than diamonds with a clarity rating of "VVS1". Let's conduct a permutation test to make sure.

In [ ]:
# First we'll calculate the mean price of each group

vvs1_mean = diamonds[diamonds['clarity'] == 'VVS1']['price'].mean()
si2_mean = diamonds[diamonds['clarity'] == 'SI2']['price'].mean()

print(vvs1_mean)
print(si2_mean)

The following function is borrowed from the [*Principles and Techniques of Data Science*, Sect. 18.1](https://www.textbook.ds100.org/ch/18/hyp_introduction.html)

In [ ]:
def shuffle(series):
    '''
    Shuffles a series and resets index to preserve shuffle when adding series
    back to DataFrame.
    '''
    
    return series.sample(frac=1, replace=False).reset_index(drop=True)

In [ ]:
# Initialize an empty array of mean differences
diffs = np.array([])

iterations = 1000
for _ in range(iterations):
    
    # Add a "shuffled" column that will contain a shuffle of diamond prices
    diamonds['Shuffled'] = shuffle(diamonds['price'])
    
    # Construct the difference in means between the two arbitrarily labeled
    # groups
    diff = diamonds[diamonds['clarity'] == 'SI2']['Shuffled'].mean() -\
    diamonds[diamonds['clarity'] == 'VVS1']['Shuffled'].mean()
    
    # Add the difference to our array of mean differences
    diffs = np.append(diffs, diff)

In [ ]:
# Get a measure of p-value by counting the number of differences that are
# greater than the difference between the real means of the two groups
p = np.count_nonzero(diffs >= si2_mean - vvs1_mean) / iterations
p

## Bootstrapping Real Data

Below we read in a dataset containing information about public toilets in Berlin.

In [ ]:
berlin = pd.read_excel('data/20191101_berlinertoiletten-2.xlsx')

In [ ]:
berlin.info()

In [ ]:
berlin['Price'].mean()

In [ ]:
means = []

for _ in range(10000):
    sample = np.random.choice(berlin['Price'].values, size=len(berlin['Price']))
    means.append(np.mean(sample))

plt.hist(means);

**Question**: To what extent could we use these results to draw inferences about public toilet prices in all of Germany? Or all of Europe? Or about past or future toilet prices?

## Bootstrapping in Linear Regression

[Here](https://www.textbook.ds100.org/ch/18/hyp_regression.html) is a great example of how we could use bootstrapping in the context of linear regression.

The basic idea is that we want some measure of the error of our coefficients. Bootstrapping to the rescue! If we fit *many* linear regressions to different bootstrapped samples of our data and calculate the coefficients each time, we can then have a distribution of coefficients that we can use as a confidence interval.

In [ ]:
np.random.seed(42)
X, Y = make_regression(n_features=2, noise=5)

In [ ]:
fig, ax = plt.subplots(2, figsize=(6, 10))
ax[0].scatter(X[:, 0], Y)
ax[1].scatter(X[:, 1], Y);

In [ ]:
lr = LinearRegression()
lr.fit(X, Y)

In [ ]:
lr.coef_

In [ ]:
df = pd.DataFrame(X, columns=('X1', 'X2'))
df_plus_y = pd.concat([df, pd.Series(Y, name='Y')], axis=1)

In [ ]:
# Now with bootstrapping

X1_coefs = []
for _ in range(1000):
    inds = np.random.choice(range(100), size=10)
    rows = df_plus_y.iloc[inds, :]
    lr = LinearRegression().fit(rows[['X1', 'X2']], rows['Y'])
    X1_coefs.append(lr.coef_[0])

In [ ]:
coefs_sorted = sorted(X1_coefs)
print(coefs_sorted[24])
print(coefs_sorted[974])

In [ ]:
X2_coefs = []
for _ in range(1000):
    inds = np.random.choice(range(100), size=10)
    rows = df_plus_y.iloc[inds, :]
    lr = LinearRegression().fit(rows[['X1', 'X2']], rows['Y'])
    X2_coefs.append(lr.coef_[1])

In [ ]:
coefs_sorted = sorted(X2_coefs)
print(coefs_sorted[24])
print(coefs_sorted[974])

### Comparison of these Results to `statsmodels`'s

In [ ]:
sm.OLS(endog=Y, exog=X).fit().summary()

### Comparison of these Results to `seaborn`'s

In [ ]:
lm1 = sns.lmplot(data=df_plus_y, x='X1', y='Y');

In [ ]:
lm2 = sns.lmplot(data=df_plus_y, x='X2', y='Y');

## Bootstrapping Challenge


Suppose we had the following two samples of automobile MPG ratings. The question is whether Group 2 (the experimental group) has a significantly higher MPG rating than Group 1 (the control group).

First, we'll make some preliminary calculations and run a hypothesis test.

In [ ]:
# Read in the files:

grp_1 = pd.read_csv('data/group1.csv', index_col=0, squeeze=True)
grp_2 = pd.read_csv('data/group2.csv', index_col=0, squeeze=True)

In [ ]:
grp_1.mean()

In [ ]:
grp_2.mean()

In [ ]:
grp_1.shape

In [ ]:
grp_2.shape

In [ ]:
grp_1.std()

In [ ]:
grp_2.std()

In [ ]:
grp_1.hist();

In [ ]:
grp_2.hist();

In [ ]:
stats.ttest_ind(grp_1, grp_2, equal_var=False)

In [ ]:
def Cohen_d(group1, group2):

    """
    Computes Cohen's d.
    """
    
    # group1: Series or NumPy array
    # group2: Series or NumPy array

    # returns a floating point number 

    diff = group1.mean() - group2.mean()

    n1 = len(group1)
    n2 = len(group2)
    var1 = group1.var(ddof=1)
    var2 = group2.var(ddof=1)

    # Calculate the pooled threshold
    pooled_var = ((n1-1) * var1 + (n2-1) * var2) / (n1 + n2 - 2)
    
    # Calculate Cohen's d statistic
    d = diff / np.sqrt(pooled_var)
    
    return d

In [ ]:
Cohen_d(grp_1, grp_2)

**Exercise**: Construct 10000 bootstrap samples of the *difference in means* between the two groups. Then order the differences and take the 250th and 9750th values to construct a 95%-confidence interval around our estimate of the difference.

In [ ]:
# Now bootstrap!

